In [35]:
from abc import ABC, abstractmethod
import requests
import time
import asyncio
import json
import websockets
import aiohttp
from datetime import datetime, timedelta
import pandas as pd

BASE_REST_SPOT_URL = "https://api.binance.com"

class Exchange(ABC):
    """
    Interface pour les échanges.
    """

    @abstractmethod
    def get_candles(self, symbol, interval, limit=500, start_date=None, end_date=None):
        pass

    @abstractmethod
    def get_available_trading_pairs(self):
        pass


class Binance(Exchange):
    """
    Classe pour interagir avec l'API de Binance.
    """

    def get_candles(self, symbol, interval, limit=500, start_date=None, end_date=None):
        base_url = "https://api.binance.com/api/v3/klines"
        params = {
            "symbol": symbol,
            "interval": interval,
            "limit": limit,
            "startTime": start_date,
            "endTime": end_date,
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Binance API error: {response.status_code} - {response.text}")

    def get_candles(self, symbol, interval, limit=1000, start_date=None, end_date=None):
        """
        Récupère des données de chandelles depuis Binance, paginées si nécessaire.
        
        :param symbol: Le symbole de trading (ex: 'BTCUSDT').
        :param interval: Intervalle des chandelles (ex: '1m', '5m', '1h', '1d').
        :param limit: Nombre maximum de chandelles par requête (par défaut: 1000).
        :param start_date: Timestamp de début (ms).
        :param end_date: Timestamp de fin (ms).
        :return: Liste complète des chandelles entre les deux dates.
        """
        base_url = "https://api.binance.com/api/v3/klines"
        all_candles = []
        current_start_time = start_date

        while current_start_time < end_date:
            params = {
                "symbol": symbol,
                "interval": interval,
                "limit": limit,
                "startTime": current_start_time,
                "endTime": end_date,
            }
            response = requests.get(base_url, params=params)
            
            if response.status_code == 200:
                candles = response.json()
                if not candles:
                    break  # Aucun résultat, fin des données
                all_candles.extend(candles)

                # Avancer le `startTime` pour la prochaine requête
                current_start_time = candles[-1][0] + 1
            else:
                raise Exception(f"Binance API error: {response.status_code} - {response.text}")

        return all_candles
    
    async def get_historical_klines(self, symbol, interval, start_time, end_time, perpetual=False):
        """
        Récupère des chandelles historiques entre start_time et end_time.

        :param symbol: Symbole de trading (ex: 'BTCUSDT').
        :param interval: Intervalle des chandelles (ex: '1m', '5m', '1h', '1d').
        :param start_time: Timestamp Unix (ms) de début.
        :param end_time: Timestamp Unix (ms) de fin.
        :param perpetual: Booléen indiquant si on utilise les données futures perpétuelles (par défaut: False).
        :return: DataFrame des chandelles.
        """
        async with aiohttp.ClientSession() as session:
            endpoint = f"{BASE_REST_SPOT_URL}/api/v3/klines" if not perpetual else f"{BASE_REST_SPOT_URL}/fapi/v1/klines"
            klines = []
            while start_time < end_time:
                params = {
                    "symbol": symbol,
                    "interval": interval,
                    "startTime": start_time,
                    "endTime": end_time,
                    "limit": 1000  # Maximum autorisé par l'API Binance
                }
                async with session.get(endpoint, params=params) as response:
                    data = await response.json()
                    if isinstance(data, list):
                        if not len(data):
                            break
                        klines.extend(data)
                        
                        # Avance le start_time à la fin de la dernière chandelle récupérée
                        start_time = data[-1][0] + 1
                        await asyncio.sleep(0.1)  # Petite pause pour éviter les limitations d'API
                    else:
                        print(data, "Error, retrying...")
                        await asyncio.sleep(5)  # Pause plus longue en cas d'erreur

            # Convertir les données en DataFrame
            df = pd.DataFrame(klines, columns=[
                'timestamp', 'open', 'high', 'low', 'close', 'volume',
                'close_time', 'quote_asset_volume', 'number_of_trades',
                'taker_buy_base', 'taker_buy_quote', 'ignore'
            ])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            df = df.drop_duplicates()
            return df
    
    def get_available_trading_pairs(self):
        base_url = "https://api.binance.com/api/v3/exchangeInfo"
        response = requests.get(base_url)
        if response.status_code == 200:
            data = response.json()
            return [symbol_info['symbol'] for symbol_info in data['symbols']]
        else:
            raise Exception(f"Binance API error: {response.status_code} - {response.text}")


class Kraken(Exchange):
    """
    Classe pour interagir avec l'API de Kraken.
    """

    def get_candles(self, symbol, interval, limit=500, start_date=None, end_date=None):
        """
        Récupère des données de chandelles depuis Kraken.
        
        :param symbol: Le symbole de trading (ex: 'XBTUSD').
        :param interval: Intervalle des chandelles (en minutes : 1, 5, 15, 30, 60, 240, 1440, etc.).
        :param limit: Non disponible sur Kraken, les chandelles retournées dépendent de la plage.
        :param start_date: Timestamp de début (en secondes).
        :param end_date: Non utilisé directement, limité par Kraken.
        :return: Liste des chandelles.
        """
        base_url = "https://api.kraken.com/0/public/OHLC"
        params = {
            "pair": symbol,
            "interval": interval,  # En minutes : 1, 5, 15, 30, 60, 240, 1440, 10080, 21600
            "since": start_date,  # En secondes (ou None pour le plus récent)
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if "error" in data and data["error"]:
                raise Exception(f"Kraken API error: {data['error']}")
            return data["result"]
        else:
            raise Exception(f"Kraken API error: {response.status_code} - {response.text}")

    def get_available_trading_pairs(self):
        """
        Récupère la liste des paires de trading disponibles sur Kraken.
        """
        base_url = "https://api.kraken.com/0/public/AssetPairs"
        response = requests.get(base_url)
        if response.status_code == 200:
            data = response.json()
            if "error" in data and data["error"]:
                raise Exception(f"Kraken API error: {data['error']}")
            return list(data["result"].keys())
        else:
            raise Exception(f"Kraken API error: {response.status_code} - {response.text}")

class CoinbasePro(Exchange):
    """
    Classe pour interagir avec l'API de Coinbase Pro.
    """

    def get_candles(self, symbol, interval, limit=300, start_date=None, end_date=None):
        """
        Récupère des données de chandelles depuis Coinbase Pro.
        
        :param symbol: Le symbole de trading (ex: 'BTC-USD').
        :param interval: Intervalle des chandelles ('1m', '5m', '15m', '1h', '6h', '1d').
        :param limit: Nombre maximum de chandelles à récupérer (300 max sur Coinbase Pro).
        :param start_date: Date de début au format ISO 8601 (ex: '2024-01-01T00:00:00Z').
        :param end_date: Date de fin au format ISO 8601.
        :return: Liste de chandelles.
        """
        base_url = "https://api.exchange.coinbase.com/products/{}/candles".format(symbol)
        params = {
            "granularity": self._interval_to_granularity(interval),
            "start": start_date,
            "end": end_date,
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Coinbase Pro API error: {response.status_code} - {response.text}")

    def get_available_trading_pairs(self):
        """
        Récupère la liste des paires de trading disponibles sur Coinbase Pro.
        """
        base_url = "https://api.exchange.coinbase.com/products"
        response = requests.get(base_url)
        if response.status_code == 200:
            data = response.json()
            return [product['id'] for product in data]
        else:
            raise Exception(f"Coinbase Pro API error: {response.status_code} - {response.text}")

    @staticmethod
    def _interval_to_granularity(interval):
        """
        Convertit un intervalle au format Coinbase Pro.
        
        :param interval: Intervalle ('1m', '5m', '15m', '1h', '6h', '1d').
        :return: Granularité en secondes.
        """
        mapping = {
            "1m": 60,
            "5m": 300,
            "15m": 900,
            "1h": 3600,
            "6h": 21600,
            "1d": 86400,
        }
        if interval in mapping:
            return mapping[interval]
        else:
            raise ValueError(f"Invalid interval: {interval}. Valid intervals: {list(mapping.keys())}")
        

class Requester:

    def __init__(self):
        self.exchange = {
            "binance": Binance(),
            "kraken": Kraken(),
            "coinbasepro": CoinbasePro(),
        }

    def get_candles(self, exchange, symbol, interval, limit=500, start_date=None, end_date=None):
        return self.exchange[exchange].get_candles(symbol, interval, limit, start_date, end_date)
    
    def get_available_trading_pairs(self, exchange):
        return self.exchange[exchange].get_available_trading_pairs()
    
    async def get_historical_klines(self, exchange, symbol, interval, start_time, end_time, perpetual=False):
        return await asyncio.run(self.exchange[exchange].get_historical_klines(symbol, interval, start_time, end_time, perpetual))

In [ ]:
import asyncio
import pandas as pd

BASE_REST_SPOT_URL = "https://api.binance.com"

async def get_historical_klines(symbol, interval, start_time, end_time, perpetual=False):
    """
    Récupère des chandelles historiques entre start_time et end_time.

    :param symbol: Symbole de trading (ex: 'BTCUSDT').
    :param interval: Intervalle des chandelles (ex: '1m', '5m', '1h', '1d').
    :param start_time: Timestamp Unix (ms) de début.
    :param end_time: Timestamp Unix (ms) de fin.
    :param perpetual: Booléen indiquant si on utilise les données futures perpétuelles (par défaut: False).
    :return: DataFrame des chandelles.
    """
    async with aiohttp.ClientSession() as session:
        endpoint = f"{BASE_REST_SPOT_URL}/api/v3/klines" if not perpetual else f"{BASE_REST_SPOT_URL}/fapi/v1/klines"
        klines = []
        while start_time < end_time:
            params = {
                "symbol": symbol,
                "interval": interval,
                "startTime": start_time,
                "endTime": end_time,
                "limit": 1000  # Maximum autorisé par l'API Binance
            }
            async with session.get(endpoint, params=params) as response:
                data = await response.json()
                if isinstance(data, list):
                    if not len(data):
                        break
                    klines.extend(data)
                    
                    # Avance le start_time à la fin de la dernière chandelle récupérée
                    start_time = data[-1][0] + 1
                    await asyncio.sleep(0.1)  # Petite pause pour éviter les limitations d'API
                else:
                    print(data, "Error, retrying...")
                    await asyncio.sleep(5)  # Pause plus longue en cas d'erreur

        # Convertir les données en DataFrame
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'number_of_trades',
            'taker_buy_base', 'taker_buy_quote', 'ignore'
        ])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df.drop_duplicates()
        return df


# Définir les paramètres
symbol = "BTCUSDT"
interval = "1m"
start_date = int(datetime(2024, 1, 1).timestamp() * 1000)
end_date = int(datetime(2024, 1, 10).timestamp() * 1000)

# Lancer la fonction asynchrone
df = asyncio.run(get_historical_klines(symbol, interval, start_date, end_date))
print(df)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [18]:
from datetime import datetime

class Kraken(Exchange):
    """
    Classe pour interagir avec l'API de Kraken.
    """

    def get_candles(self, symbol, interval, start_date=None, end_date=None):
        """
        Récupère des données de chandelles depuis Kraken, paginées si nécessaire.
        
        :param symbol: Le symbole de trading (ex: 'XBTUSD').
        :param interval: Intervalle des chandelles (en minutes : 1, 5, 15, 30, 60, 240, 1440, etc.).
        :param start_date: Timestamp de début (en secondes Unix).
        :param end_date: Timestamp de fin (en secondes Unix).
        :return: Liste complète des chandelles entre les deux dates.
        """
        base_url = "https://api.kraken.com/0/public/OHLC"
        all_candles = []
        current_since = start_date

        while current_since < end_date:
            params = {
                "pair": symbol,
                "interval": interval,  # Intervalle en minutes (1, 5, 15, 30, 60, etc.)
                "since": current_since,  # Timestamp de début (en secondes Unix)
            }
            response = requests.get(base_url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                if "error" in data and data["error"]:
                    raise Exception(f"Kraken API error: {data['error']}")
                
                candles = data["result"].get(symbol, [])
                if not candles:
                    break  # Aucun résultat, fin des données
                all_candles.extend(candles)

                # Avancer le `since` pour la prochaine requête
                current_since = candles[-1][0]  # Le dernier timestamp des données
            else:
                raise Exception(f"Kraken API error: {response.status_code} - {response.text}")

        return all_candles
    
    def get_available_trading_pairs(self):
        pass


# Exemple d'utilisation
if __name__ == "__main__":
    kraken = Kraken()

    # Convertir les dates au format timestamp (en secondes)
    start_date = int(datetime(2024, 1, 1).timestamp())
    end_date = int(datetime(2024, 1, 2).timestamp())

    try:
        candles = kraken.get_candles(
            symbol="BTCUSDT",
            interval=60,  # 1 heure
            start_date=start_date,
            end_date=end_date,
        )
        print(f"Nombre total de chandelles récupérées : {len(candles)}")
        print(f"Première chandelle : {candles[0]}")
        print(f"Dernière chandelle : {candles[-1]}")
    except Exception as e:
        print(f"Erreur : {e}")

Nombre total de chandelles récupérées : 0
Erreur : list index out of range


In [29]:
req = Requester()

start_date = int(datetime(2024, 1, 1).timestamp() * 1000)
end_date = int(datetime(2024, 1, 10).timestamp() * 1000)


# Exemple d'utilisation
candles = req.get_candles("binance", "BTCUSDT", "1m", start_date=start_date, end_date=end_date)

print(f"Nombre total de chandelles récupérées : {len(candles)}")



Nombre total de chandelles récupérées : 12961


In [36]:
req = Requester()

start_date = int(datetime(2024, 1, 1).timestamp() * 1000)
end_date = int(datetime(2024, 1, 10).timestamp() * 1000)

# Exemple d'utilisation
klines = req.get_historical_klines("binance", "BTCUSDT", "1m", start_date, end_date)

/var/folders/vk/z0p4r5z97d99dp7mpbt9yd1w0000gn/T/ipykernel_75318/1603106136.py:7: RuntimeWarning: coroutine 'Requester.get_historical_klines' was never awaited
  klines = req.get_historical_klines("binance", "BTCUSDT", "1m", start_date, end_date)


In [37]:
klines

<coroutine object Requester.get_historical_klines at 0x12a09afc0>

In [23]:
pd.DataFrame(candles)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1704063600000,42257.89000000,42291.10000000,42196.61000000,42277.25000000,114.76075000,1704063659999,4847771.68726980,2898,60.52820000,2557031.14309600,0
1,1704063660000,42277.25000000,42296.60000000,42241.10000000,42252.14000000,45.96533000,1704063719999,1943222.46387270,1919,21.35249000,902622.11714080,0
2,1704063720000,42252.15000000,42297.18000000,42252.14000000,42294.04000000,44.10371000,1704063779999,1864538.59431090,1789,27.95499000,1181804.09847090,0
3,1704063780000,42294.04000000,42345.64000000,42294.03000000,42345.62000000,45.99365000,1704063839999,1946386.77870010,1408,18.87768000,798844.25459870,0
4,1704063840000,42345.63000000,42345.65000000,42282.00000000,42282.01000000,47.50271000,1704063899999,2010082.43878790,1296,12.90190000,545913.63291830,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1436,1704149760000,43493.31000000,43501.10000000,43493.31000000,43501.10000000,18.01047000,1704149819999,783348.91858860,407,4.90100000,213174.29755060,0
1437,1704149820000,43501.09000000,43506.86000000,43501.09000000,43505.82000000,7.09761000,1704149879999,308776.30085840,299,6.17548000,268658.45582960,0
1438,1704149880000,43505.83000000,43529.94000000,43505.82000000,43529.93000000,11.46603000,1704149939999,499019.75430200,517,7.57527000,329666.30545070,0
1439,1704149940000,43529.93000000,43529.94000000,43529.93000000,43529.93000000,8.37419000,1704149999999,364527.95184690,318,4.73402000,206071.60655880,0


# Binance

In [51]:
base_url = "http://localhost:8000/binance/klines?symbol=BTCUSDT&interval=1m&start_date=2024-01-01T00:00:00&end_date=2024-01-10T00:00:00"

response = requests.get(base_url)

pd.DataFrame(response.json()['klines'])

KeyError: 'klines'

In [49]:
response.json()

{'error': 'unconverted data remains: T00:00:00Z'}

,0,1,2,3,4,5,6,7,8,9,10,11
0,1704063600000,42257.89000000,42291.10000000,42196.61000000,42277.25000000,114.76075000,1704063659999,4847771.68726980,2898,60.52820000,2557031.14309600,0
1,1704063660000,42277.25000000,42296.60000000,42241.10000000,42252.14000000,45.96533000,1704063719999,1943222.46387270,1919,21.35249000,902622.11714080,0
2,1704063720000,42252.15000000,42297.18000000,42252.14000000,42294.04000000,44.10371000,1704063779999,1864538.59431090,1789,27.95499000,1181804.09847090,0
3,1704063780000,42294.04000000,42345.64000000,42294.03000000,42345.62000000,45.99365000,1704063839999,1946386.77870010,1408,18.87768000,798844.25459870,0
4,1704063840000,42345.63000000,42345.65000000,42282.00000000,42282.01000000,47.50271000,1704063899999,2010082.43878790,1296,12.90190000,545913.63291830,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12956,1704840960000,46081.10000000,46139.74000000,46020.00000000,46107.96000000,111.54905000,1704841019999,5142196.63377540,3992,53.80575000,2480302.64917820,0
12957,1704841020000,46107.96000000,46258.16000000,46104.00000000,46192.78000000,109.44526000,1704841079999,5056120.55716240,4677,62.77735000,2899838.54131730,0
12958,1704841080000,46192.79000000,46261.09000000,46185.03000000,46248.00000000,67.66290000,1704841139999,3127792.82015870,2183,41.63539000,1924308.74112360,0
12959,1704841140000,46247.99000000,46261.09000000,46212.25000000,46254.00000000,71.13069000,1704841199999,3289504.50001080,2124,27.03753000,1250202.57229790,0


In [5]:
import pandas as pd

df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                    'quote_asset_volume', 'number_of_trades', 'buy_volume', 'buy_notional',
                                    'ignore'])
df = df.drop_duplicates()
df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='ms')
df.set_index('timestamp', inplace=True)
df['volume'] = df['volume'].astype(float)
df['notional'] = df['quote_asset_volume'].astype(float)


In [7]:
df

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,buy_volume,buy_notional,ignore,notional
timestamp,,,,,,,,,,,,
2024-12-19 06:00:00,101300.01000000,101392.15000000,101015.44000000,101200.01000000,712.40267,1734591599999,72100946.41976840,158148,351.76521000,35602239.95190680,0,7.210095e+07
2024-12-19 07:00:00,101200.01000000,101727.28000000,101124.46000000,101656.57000000,716.67385,1734595199999,72740307.57167420,136868,421.81834000,42818366.56927790,0,7.274031e+07
2024-12-19 08:00:00,101656.57000000,101820.00000000,101380.10000000,101640.40000000,745.81150,1734598799999,75795288.13411030,137730,340.61439000,34618104.87013000,0,7.579529e+07
2024-12-19 09:00:00,101640.40000000,101873.08000000,101537.56000000,101815.99000000,863.07944,1734602399999,87802406.99781640,113454,499.66043000,50828969.60350750,0,8.780241e+07
2024-12-19 10:00:00,101815.99000000,102490.00000000,101758.19000000,102488.00000000,1203.29225,1734605999999,122920516.40468990,126174,657.97384000,67222241.95052680,0,1.229205e+08


In [ ]:
candles

# Kraken

In [ ]:
# Utilisation de Kraken
kraken = Kraken()
try:
    kraken_pairs = kraken.get_available_trading_pairs()
    print(f"Kraken pairs (total: {len(kraken_pairs)}): {kraken_pairs[:10]}")

    # Exemple pour récupérer des chandelles de Kraken
    candles = kraken.get_candles(symbol="XBTUSD", interval=5, start_date=1680000000)
    #print("Exemple de chandelles Kraken :", candles)
except Exception as e:
    print(f"Erreur Kraken : {e}")

# Coinbase Pro

In [ ]:
coinbase = CoinbasePro()
try:
    trading_pairs = coinbase.get_available_trading_pairs()
    print(f"Coinbase Pro pairs (total: {len(trading_pairs)}): {trading_pairs[:10]}")

    # Exemple pour récupérer des chandelles
    candles = coinbase.get_candles(
        symbol="BTC-USD",
        interval="1h",   
        start_date="2024-01-01T00:00:00Z",
        end_date="2024-01-02T00:00:00Z",
    )

    #print(f"Exemple de chandelles Coinbase Pro : {candles[:5]}")
except Exception as e:
    print(f"Erreur Coinbase Pro : {e}")